# JSON to CSV data conversion

This Jupyter Notebook is part of the time travel distance project. This notebook merges the different datsets queried from the Google API into one csv file. It also drops additional query information and adds the closest elevation information that is avaliable in the elevation dataset.

To estimate horizontal distances we use the geodesic distance approximation provided by the geopy package. 

## Import

In [11]:
import numpy as np
import pandas as pd
import json
from geopy import distance as distance_calculator
from scipy.spatial import KDTree
import random
import math

## Define paths

In [12]:
main_path = 'data/'
data_files = ['driving.json', 'walking.json', 'bicycling.json']
modes = ['driving', 'walking', 'bicycling']

## Load elevation data

In [13]:
# empty frame
elevation_frame = pd.DataFrame({})

# read elevation data
with open(main_path + 'elevation.json', 'r') as f:
    data = json.load(f)

# fill frame
for query in data:
    dict = query[0]

    elevation_frame = elevation_frame.append({
      'lat': dict['location']['lat'],
      'lng': dict['location']['lng'],
      'elevation': dict['elevation']
    }, ignore_index=True)

# construct KD tree for efficient nearest neighbour search
elevation_kdtree = KDTree(np.stack((elevation_frame.lat.to_numpy(),
                                    elevation_frame.lng.to_numpy()), axis=1))


In [14]:
def get_closest_elevation(location):
    
    d, i = elevation_kdtree.query((location['lat'], location['lng']))
    return elevation_frame.elevation[i]

## Extract information from json and add to dataframe

Loop through all datasets (one per mean of transport) and all entries (one per location) to:
- extract start and target location
- compute horizontal distance
- find closest elevation point
- compute vertical distances

In [24]:
df = pd.DataFrame({})

for data_file, mode in zip(data_files, modes):
    
    with open(main_path + data_file, 'r') as f:
        data = json.load(f)

    start_elevation = get_closest_elevation(data[0][0]['legs'][0]['start_location'])

    for query in data:
        dict = query[0]['legs'][0]
        
        # get horizontal difference in meters
        distance_horizontal = round(distance_calculator.distance(dict['start_location'].values(), dict['end_location'].values()).km*1000)

        # get vertical difference in meters
        distance_vertical = get_closest_elevation(dict['end_location']) - start_elevation
        
        # positive vertical difference in meters
        distance_vertical_pos = max(0, distance_vertical)
       
        # negative vertical difference in meters
        distance_vertical_neg = abs(min(0, distance_vertical))
        
        # absolute vertical difference in meters
        distance_vertical_abs = abs(distance_vertical)
        
        df = df.append({
          'start_location_lat': dict['start_location']['lat'],
          'start_location_lng': dict['start_location']['lng'],
          'end_location_lat': dict['end_location']['lat'],
          'end_location_lng': dict['end_location']['lng'],
          'distance_path': dict['distance']['value'],
          'distance_horizontal': distance_horizontal,
          'distance_vertical': distance_vertical,
          'distance_vertical_pos': distance_vertical_pos,
          'distance_vertical_neg': distance_vertical_neg,
          'distance_vertical_abs': distance_vertical_abs,
          'duration': dict['duration']['value'],
          'mode': mode
          }, ignore_index=True)

337.0936584472656
337.0936584472656
337.0936584472656


## Preview

In [25]:
df.head()

,distance_horizontal,distance_path,distance_vertical,distance_vertical_abs,distance_vertical_neg,distance_vertical_pos,duration,end_location_lat,end_location_lng,mode,start_location_lat,start_location_lng
0,4216.0,8608.0,101.332123,101.332123,0.0,101.332123,941.0,48.494872,9.095148,driving,48.520281,9.052807
1,4149.0,8152.0,97.190277,97.190277,0.0,97.190277,919.0,48.496965,9.096650,driving,48.520281,9.052807
2,3969.0,8509.0,99.422424,99.422424,0.0,99.422424,886.0,48.498032,9.094813,driving,48.520281,9.052807
3,3955.0,8235.0,99.761536,99.761536,0.0,99.761536,925.0,48.498713,9.095367,driving,48.520281,9.052807
4,3821.0,7803.0,96.327454,96.327454,0.0,96.327454,815.0,48.500307,9.094886,driving,48.520281,9.052807


## Store

In [26]:
df.to_csv(main_path + 'combined_data.csv', index=False)